In [66]:
import yahoo_fin.stock_info as si
import pandas as pd

In [67]:
# get ticker from user
ticker = input("Enter Ticker of a company you want to check")

In [68]:
# Load financial data

financials = si.get_financials(ticker)
quarterly_income_statement = pd.DataFrame(financials['quarterly_income_statement'])
quarterly_balance_sheet = pd.DataFrame(financials['quarterly_balance_sheet'])
quarterly_cash_flow = pd.DataFrame(financials['quarterly_cash_flow'])

In [69]:
# Clean financial data

financials_df = pd.concat([quarterly_income_statement, quarterly_balance_sheet, quarterly_cash_flow])
financials_df.drop_duplicates(inplace=True)
financials_df = financials_df.transpose()
financials_df

Breakdown,researchDevelopment,effectOfAccountingCharges,incomeBeforeTax,netIncome,sellingGeneralAdministrative,grossProfit,ebit,interestExpense,incomeTaxExpense,totalRevenue,...,changeInCash,repurchaseOfStock,effectOfExchangeRate,totalCashFromOperatingActivities,depreciation,changeToInventory,changeToAccountReceivables,changeToNetincome,capitalExpenditures,otherCashflowsFromFinancingActivities
endDate,,,,,,,,,,,,,,,,,,,,,
2022-09-30,NaN,None,148588000,112775000,265795000,382943000,117148000,-641000,35813000,823303000,...,255198000.0,-18870000.0,1530000.0,296381000.0,17092000.0,-11126000.0,64780000.0,38758000.0,-27811000.0,NaN
2022-06-30,347196000.0,None,229378000,161432000,252586000,858217000,258435000,-678000,67946000,911397000,...,178566000.0,-4693000.0,988000.0,174157000.0,16107000.0,-23287000.0,-120624000.0,111451000.0,-11743000.0,NaN
2022-03-31,347196000.0,None,229378000,0,208859000,332506000,123647000,-678000,0,0,...,199299000.0,-2161000.0,210000.0,215741000.0,16438000.0,-13903000.0,53956000.0,101005000.0,-17006000.0,-13473000.0
2021-12-31,347196000.0,None,120020000,563851000,226556000,338609000,112053000,-752000,-443831000,862853000,...,58457000.0,-29940000.0,148000.0,115351000.0,17558000.0,-4750000.0,-99611000.0,-426404000.0,-34463000.0,-13473000.0


In [70]:
# obtain Market Value

quote = si.get_quote_table(ticker)
quote['Market Cap']

multipliers = {'K':1000, 'M':1000000, 'B':1000000000}

def string_to_int(string):
    if string[-1].isdigit():
        return int(string)
    mult = multipliers[string[-1]]
    return int(float(string[:-1]) * mult)

market_value = string_to_int(quote['Market Cap'])

c:\Users\yutak\Anaconda3\envs\dev\lib\site-packages\yahoo_fin\stock_info.py:295: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = tables[0].append(tables[1])
c:\Users\yutak\Anaconda3\envs\dev\lib\site-packages\yahoo_fin\stock_info.py:302: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append(quote_price)


In [76]:
# Create testing dataset

X_test = pd.DataFrame()
X_test['current_assets'] = financials_df['totalCurrentAssets']
X_test['cost_of_goods_sold'] = financials_df['totalOperatingExpenses']
X_test['depreciation_and_amortization'] = financials_df['depreciation'] + financials_df['intangibleAssets']
X_test['EBITDA'] = financials_df['grossProfit'] - financials_df['totalOperatingExpenses'] + financials_df['depreciation'] + financials_df['intangibleAssets']
X_test['inventory'] = financials_df['inventory']
X_test['net_income'] = financials_df['netIncome']
X_test['total_receivables'] = financials_df['netReceivables']
X_test['market_value'] = market_value
X_test['net_sales'] = financials_df['totalRevenue']
X_test['total_assets'] = financials_df['totalAssets']
X_test['total_long_term_debt'] = financials_df['totalLiab']
X_test['EBIT'] = financials_df['grossProfit'] - financials_df['totalOperatingExpenses']
X_test['gross_profit'] = financials_df['grossProfit']
X_test['total_current_liabilities'] = financials_df['totalCurrentLiabilities']
X_test['retained_earnings'] = financials_df['retainedEarnings']
X_test['total_revenue'] = financials_df['totalRevenue']
X_test['total_liabilities'] = financials_df['totalCurrentLiabilities']
X_test['total_operationg_expenses'] = financials_df['totalOperatingExpenses']

# get the most-recent and valid column
X_test.dropna(inplace=True)

In [77]:
# Apply trained models and make prediction

from joblib import load

X_scaler = load('scaler.joblib')
rfc_model = load('randomforestclassifier.joblib')

X_test_scaled = X_scaler.transform(X_test)
prediction = rfc_model.predict(X_test_scaled)
most_recent_result = prediction[0]

In [84]:
if most_recent_result == 0:
    print(f"According to the most recent financial stats, {ticker} will not bankrupt.")
elif most_recent_result == 1:
    print(f"According to the most recent financial stats, {ticker} is facing financial difficulty")

According to the most recent financial stats, INCY will not bankrupt.
